In [1]:
import os
path = os.path.abspath(os.path.join('..','..'))
import sys
sys.path.append(path)
import pandas as pd
import numpy as np
from reservoirpy.materialbalancepy import mbe as mb
from reservoirpy.pvtpy import black_oil as bl

In [2]:
data = np.array([
    [0.1000000, 0.0000000, 1.0000000, 0],
    [0.2000000, 0.0138408, 0.6869530, 0],
    [0.3000000, 0.0553633, 0.4471809, 0],
    [0.4000000, 0.1245675, 0.2709139, 0],
    [0.5000000, 0.2214533, 0.1483818, 0],
    [0.6000000, 0.3460208, 0.0698148, 0],
    [0.7000000, 0.4982699, 0.0254427, 0],
    [0.8000000, 0.6782007, 0.0054956, 0],
    [0.9000000, 0.8858131, 0.0002035, 0],
    [0.9500000, 1.0000000, 0.0000000, 0],
    [1.0000000, 1.0000000, 0.0000000, 0],
])
kr = mb.kr(data,columns=['sw','krw','kro','pc'])


In [3]:
# Oil PVT
oil_pvt = bl.pvt({
    'pressure': [2925,2100,1800,1500],
    'bo':[1.429,1.480,1.468,1.440],
    'bt':[1.429,1.480,1.559,1.792],
    'rs':[1340,1340,1280,1150],
    'bg':[0.001,0.001283,0.001518,0.00183],
    'muo':[10,7,7.5,8],
    'mur':[0,34.1,38.3,42.4]
})

formation = 'fm1'
api = 25
pb = 2100
rsi = 1340
temp = 175

o = bl.oil(
    formation=formation,
    api=api,
    pb=pb,
    rsb=rsi,
    temp=temp,
    pvt=oil_pvt
)

In [4]:
# Water

w = bl.water(
    formation=formation,
    pb = pb,
    temp = temp)

w.pvt_from_correlations(start_pressure=1000,end_pressure=3000,n=5)
w.pvt

,rsw,cw,bw,muw,rhow
pressure,,,,,
1000.0,6.803417,0.000003,1.029221,0.375964,60.628376
1500.0,8.975257,0.000003,1.028531,0.375964,60.669053
2000.0,11.044822,0.000003,1.027709,0.375964,60.717562
2500.0,13.012112,0.000003,1.026144,0.375964,60.810184
3000.0,14.877126,0.000003,1.024409,0.375964,60.913157


In [5]:
swi=0.15
cf=4.9e-6
ooip=10e6

In [6]:
res1 = mb.reservoir(
    n=ooip,
    swi=swi,
    cf = cf,
    oil = o,
    water = w,
    pi=2925,    
    kr_wo = kr
)


In [7]:
r = np.linspace(2100,2900,10)[::-1]

r

array([2900.        , 2811.11111111, 2722.22222222, 2633.33333333,
       2544.44444444, 2455.55555556, 2366.66666667, 2277.77777778,
       2188.88888889, 2100.        ])

In [11]:
res1.forecast_np(r,wp=True)

,np,gp,wp,wor,sw
2900.000000,10729.647023,1.437773e+07,2320.232275,0.216245,0.150000
2811.111111,48929.681186,6.556577e+07,10234.134452,0.209160,0.149994
2722.222222,87207.237566,1.168577e+08,17624.374549,0.202098,0.149992
2633.333333,125551.519420,1.682390e+08,24505.411003,0.195182,0.150021
2544.444444,163954.472050,2.196990e+08,30888.034166,0.188394,0.150082
2455.555556,202409.723876,2.712290e+08,36780.924689,0.181715,0.150172
2366.666667,240912.348797,3.228225e+08,42190.707532,0.175129,0.150292
2277.777778,279458.771082,3.744748e+08,47122.015958,0.168619,0.150442
2188.888889,318046.876733,4.261828e+08,51577.570745,0.162170,0.150620
2100.000000,356675.884617,4.779457e+08,55558.214914,0.155767,0.150827


4.9e-06

344656.3891891892